# Voting Mechanism Simulation Examples

Our goal here is to show basic use cases for the code, and do basic proof-of-concept on possible future extensions.  

In [ ]:
# Standard Imports
import numpy as np
import pandas as pd
import os
from random import choice
from typing import Dict

import sys
sys.path.append('..')  # Add this line to include the directory above

# Custom imports
from custom_types import UserNFTs
from mechanisms.single_choice_weighted_plurality import SingleChoiceWeightedPlurality

In [ ]:
file_name = "../data/nft_data_may_282024__cleaned.csv"
file_exists = os.path.exists(file_name)
file_exists

# Key Question

How often would a vote that incorporates credibility-weighting by NFTs produce a different outcome than simply doing one wallet, one vote? We use the actual TE Academy NFT data as of May 28 to look at this question. 

## Inputting Sample Data as a Dictionary

In [ ]:
VOTER_DATA_FILENAME = "../data/nft_data_may_28_2024_cleaned.csv"

voter_data = pd.read_csv(VOTER_DATA_FILENAME)

# We may need to drop a specific column. 
voter_data.drop(columns = ['Unnamed: 0'], 
                inplace = True)
voter_data.set_index('ID', inplace = True)

In [ ]:
sample_voters = voter_data.to_dict(orient='index')


In [ ]:
sample_voters

In [ ]:
voters = {key: UserNFTs(sample_voters.get(key))
          for key, _ in sample_voters.items()
          }

In [ ]:
voters

In [ ]:
# Get the first voter
first_voter = list(sample_voters.values())[0]

NFT_weights = {key: 1.0 
               for key
               in first_voter.keys()
               }

In [ ]:
def calc_voter_weights_from_NFT_weight(voters: Dict,
                                       nft_weights: Dict) -> Dict:
    new_dict = {}
    for voter in voters.keys():
        new_dict[voter] = {}
        new_dict[voter]["weight"] = 0
        for nft_name, nft_val in nft_weights.items():
            if voters.get(voter).get(nft_name):
                new_dict[voter]["weight"] += nft_val

    return new_dict

In [ ]:
voter_weights = calc_voter_weights_from_NFT_weight(voters,
                                                   NFT_weights)

In [ ]:
voter_weights

In [ ]:
uniform_weights = {key: {"weight": 1.0} for key in sample_voters.keys()}

In [ ]:
uniform_weights

In [14]:
SCWPCalculator = SingleChoiceWeightedPlurality()

NUM_EXPERIMENTS = 100_000

# Create an empty DataFrame to store simulation results
results_list = [] 

for k in range(NUM_EXPERIMENTS):
    sample_choices = {
                   key: choice(["A","B","C","D"])
                   for key in sample_voters.keys()
                 }
    weighted_winner, weighted_results  = SCWPCalculator.calculate(voter_weights,
                            sample_choices)
    uniform_winner, uniform_results = SCWPCalculator.calculate(uniform_weights,
                            sample_choices)
    # Create an empty DataFrame to store simulation results
    results_list.append({'Experiment': k,
                        'weighted_winner': weighted_winner, 
                        'weighted_candidate_scores': weighted_results,
                        'uniform_winner': uniform_winner,
                        'uniform_candidate_scores': uniform_results}
                        )


results_df = pd.DataFrame(results_list)

#TODO: Refactor for speed if needed.  

    

In [ ]:
results_df

In [ ]:
(results_df['weighted_winner'] == results_df['uniform_winner']).mean()

In [ ]:
import mechanisms.group_hug_mechanism as GH